In [1]:
import setup
setup.init_django()

In [2]:
from market.models import StockQuote

In [3]:
from django.db.models import Avg, F, RowRange, Window, Max
from django.db.models.functions import TruncDate
from django.utils import timezone
from datetime import timedelta
from decimal import Decimal

In [4]:
days_ago = 30
now = timezone.now()
start_date = now - timedelta(days=30)
end_date = now

# filtering data
qs = StockQuote.objects.filter(time__range=(start_date, end_date))
qs.count()

4336

In [5]:
total = 0
for obj in qs:
    total += obj.close_price

In [6]:
total /qs.count()

Decimal('219.3231125230627306273062731')

In [7]:
# average prices for entire month
qs.values('company').annotate(avg_price=Avg('close_price'))

<QuerySet [{'company': 2, 'avg_price': Decimal('170.1696036020583190')}, {'company': 1, 'avg_price': Decimal('226.9587362110311751')}]>

In [8]:
count = 5
ticker = "AAPL"
rolling_qs = list(qs.filter(company__ticker = ticker).order_by('time')[:count]) # grabbing orginal window
rolling_qs.reverse() # orginal time order

In [9]:
rolling_qs

[<StockQuote: StockQuote object (313490)>,
 <StockQuote: StockQuote object (313489)>,
 <StockQuote: StockQuote object (313488)>,
 <StockQuote: StockQuote object (313487)>,
 <StockQuote: StockQuote object (313486)>]

In [10]:
total = 0
for i, obj in enumerate(rolling_qs):
    total += obj.close_price
    avg = total / (i + 1)
    print(i + 1, obj.id, obj.close_price, avg)

1 313490 234.2950 234.2950
2 313489 234.3950 234.3450
3 313488 234.4400 234.3766666666666666666666667
4 313487 234.2203 234.337575
5 313486 234.4180 234.35366


In [16]:
latest_daily_timestamps = (
    StockQuote.objects.filter(company__ticker = ticker, time__range=(start_date, end_date))
    .annotate(date=TruncDate('time'))
    .values('company', 'date')
)

latest_daily_timestamps

In [17]:
qs = StockQuote.objects.filter(company__ticker = ticker, time__range=(start_date, end_date))

In [12]:
frame_start = -(count - 1)
ma_val = qs.annotate(
        ma = Window(
            expression=Avg('close_price'),
            order_by=F('time').asc(),
            partition_by=[],
            frame = RowRange(start=frame_start, end=0), # based on amount we want to get
        )
).order_by('-time')

In [13]:
for obj in ma_val[:5]:
    print(obj.id, obj.close_price, obj.ma, obj.time)

330238 235.5300 235.5260000000000000 2024-11-27 00:55:00+00:00
330237 235.3600 235.5400000000000000 2024-11-27 00:50:00+00:00
330236 235.4900 235.5833600000000000 2024-11-27 00:45:00+00:00
330235 235.6000 235.6113600000000000 2024-11-27 00:40:00+00:00
330234 235.6500 235.6153600000000000 2024-11-27 00:35:00+00:00


In [14]:
frame_start = -(count - 1)
ma_vals = qs.annotate(
        ma_5 = Window( # 5 minute intervals of 5
            expression=Avg('close_price'),
            order_by=F('time').asc(),
            partition_by=[],
            frame = RowRange(start=-4, end=0), # based on amount we want to get
        ),
        ma_20 = Window( # 20 minute intervals of 5
            expression=Avg('close_price'),
            order_by=F('time').asc(),
            partition_by=[],
            frame = RowRange(start=-19, end=0), # based on amount we want to get
        )
).order_by('-time').first()

ma_vals.id, ma_vals.close_price, ma_vals.ma_5, ma_vals.ma_20

(330238,
 Decimal('235.5300'),
 Decimal('235.5260000000000000'),
 Decimal('235.5600550000000000'))